<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/SGDClassifier_with_full_data_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir

files = listdir('/content/drive/MyDrive/amex-default-prediction/data/raw/')

In [2]:
import gc 

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [3]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
TRAIN_LABELS_PATH = '/content/train_labels.csv'

In [4]:
!unzip /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip \
 train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip
replace train_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_labels.csv        


In [5]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [6]:
def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [7]:
flag = True
X = pd.DataFrame()
Y = pd.DataFrame()

for f in files:
  print(f)
  df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/raw/' + f)
  df_tmp = append_label(df_tmp)


  if(flag):
     enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
     df_tmp[['D_63', 'D_64']]=enc.fit(df_tmp[['D_63', 'D_64']])
     flag = False

  df_tmp[['D_63', 'D_64']]=enc.transform(df_tmp[['D_63', 'D_64']])
  df_tmp.fillna(0, inplace=True)

  X= X.append(df_tmp.loc[:, ~df_tmp.columns.isin(['customer_ID', 'S_2', 'target'])])

  Y = Y.append(df_tmp.loc[:, df_tmp.columns == 'target'])

  
  del df_tmp
  gc.collect()


data_2017-04.parquet
data_2017-05.parquet
data_2017-06.parquet
data_2017-03.parquet
data_2017-07.parquet
data_2017-08.parquet
data_2017-09.parquet
data_2017-10.parquet
data_2017-11.parquet
data_2017-12.parquet
data_2018-01.parquet
data_2018-02.parquet
data_2018-03.parquet


In [8]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
gc.collect()

328

In [11]:
del X, Y

In [12]:
gc.collect()

252

In [13]:
clf = SGDClassifier(verbose=1000)

In [14]:
clf.fit(X_train, y_train.values.ravel())

-- Epoch 1
Norm: 4.17, NNZs: 188, Bias: -0.067607, T: 4425160, Avg. loss: 1.144776
Total training time: 4.60 seconds.
-- Epoch 2
Norm: 4.14, NNZs: 188, Bias: -0.034551, T: 8850320, Avg. loss: 0.438999
Total training time: 9.21 seconds.
-- Epoch 3
Norm: 4.08, NNZs: 188, Bias: 0.019951, T: 13275480, Avg. loss: 0.378818
Total training time: 14.37 seconds.
-- Epoch 4
Norm: 3.84, NNZs: 188, Bias: 0.014448, T: 17700640, Avg. loss: 0.375021
Total training time: 19.17 seconds.
-- Epoch 5
Norm: 3.87, NNZs: 188, Bias: 0.012762, T: 22125800, Avg. loss: 0.344313
Total training time: 23.87 seconds.
-- Epoch 6
Norm: 3.77, NNZs: 188, Bias: 0.025896, T: 26550960, Avg. loss: 0.333284
Total training time: 28.58 seconds.
-- Epoch 7
Norm: 3.85, NNZs: 188, Bias: 0.008350, T: 30976120, Avg. loss: 0.334590
Total training time: 33.26 seconds.
-- Epoch 8
Norm: 3.79, NNZs: 188, Bias: 0.021113, T: 35401280, Avg. loss: 0.334738
Total training time: 37.92 seconds.
-- Epoch 9
Norm: 3.71, NNZs: 188, Bias: 0.018697, 

SGDClassifier(verbose=1000)

In [15]:
clf.score(X_test, y_test)

0.8730487728816378

In [16]:
clf.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 1000,
 'warm_start': False}